In [1]:
import os
import jieba
import pandas as pd
from utils import files_processing 
import multiprocessing
from gensim.models import word2vec

In [2]:
# 源文件所在目录
source_folder = 'F:/BI/第四周/名企班/L4/word2vec/three_kingdoms/source'
segment_folder = 'F:/BI/第四周/名企班/L4/word2vec/three_kingdoms/segment'

In [3]:
file_list = files_processing.get_files_list(source_folder, postfix='*.txt')

In [5]:
with open(file_list[0], 'r') as f:
    document = f.read()

In [6]:
document[:10]

'三国演义\n作者：罗贯'

In [7]:
# 字词分割，对整个文件内容进行字词分割
def segment_lines(file_list, segment_out_dir, stopwords=[]):
    """从source中读取文本，分词后，保存到segment目录中"""
    for i,file in enumerate(file_list):
        # 用于存储分词结果的路径
        segment_out_name = os.path.join(segment_out_dir, 'segment_{}.txt'.format(i))
        with open(file, 'r') as f:
            document = f.read()
            # jieba分词
            document_cut = jieba.cut(document)
            sentence_segment=[]
            for word in document_cut:
                if word not in stopwords:
                    sentence_segment.append(word)
            result = ' '.join(sentence_segment)
            with open(segment_out_name, 'w') as f2:
                f2.write(result)

In [8]:
segment_lines(file_list, segment_folder)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/dk/4d4_y1kn1js69jdc1n1xyjtr0000gn/T/jieba.cache
Loading model cost 0.710 seconds.
Prefix dict has been built successfully.


In [9]:
# 相似度

In [10]:
sentences = word2vec.PathLineSentences(segment_folder)

In [11]:
# 建立模型

In [12]:
model = word2vec.Word2Vec(sentences, 
                          size=100,  # 向量维度，默认为100
                          window=3,  # 句子中当前单词和被预测单词的最大距离
                          min_count=1) # 需要训练词语的最小出现次数，默认为5

In [15]:
# 计算相似度

In [16]:
similars10 = model.wv.most_similar(positive=['曹操'], topn=10)

In [17]:
similars10

[('关公', 0.9948543906211853),
 ('孔明亦', 0.9942571520805359),
 ('众官', 0.9936116933822632),
 ('彰', 0.992612361907959),
 ('孔明', 0.9916199445724487),
 ('先主', 0.9904195070266724),
 ('相害', 0.9903540015220642),
 ('请', 0.9899086952209473),
 ('维问', 0.9895586371421814),
 ('哀痛', 0.9894794225692749)]

In [21]:
# 曹操+刘备-张飞

In [13]:
model.wv.similarity('曹操', '刘备')

0.98375

In [19]:
model.wv.similarity('曹操', '张飞')

0.9841915

In [20]:
model.wv.similarity('刘备', '张飞')

0.9587526

In [18]:
model.wv.most_similar(positive=['曹操', '刘备'], negative=['张飞'])

[('丞相', 0.9932718276977539),
 ('主公', 0.9924185872077942),
 ('臣', 0.992241382598877),
 ('今番', 0.991971492767334),
 ('此', 0.9919576644897461),
 ('商议', 0.9906771183013916),
 ('哀告', 0.9903850555419922),
 ('吾', 0.9900258779525757),
 ('今', 0.9899616837501526),
 ('玄德公', 0.9898769855499268)]